# CITEseq data analysis
*Author: Lena Boehme, Taghon lab, 2023*

## Integrated RNA/protein analysis

In order to best distinguish developing thymocyte stages and subtypes, we ideally use both RNA and ADT information. We therefore build an integrated UMAP and supervised PCA, which takes both modalities into account. This is based on the seurat [Weighted Nearest Neighbour approach](https://www.sciencedirect.com/science/article/pii/S0092867421005833?via%3Dihub).

### Setup

In [ ]:
options(repr.plot.width=12, repr.plot.height=6)

options(scipen=100) #avoid scientific notation of numbers

library(SeuratDisk)
library(Seurat)
library(matrixStats)
library(ggplot2)
library(pheatmap)
library(reshape2)
library(dplyr)
library(tidyr)
library(viridis)
library(RColorBrewer)
library(stringr)

library(batchelor)
library(BiocParallel)
library(BiocNeighbors)

In [ ]:
sessionInfo()

In [ ]:
pal12 <- colorRampPalette(brewer.pal(12, "Paired"))(12)
pal24 <- colorRampPalette(brewer.pal(12, "Paired"))(24)
pal36 <- colorRampPalette(brewer.pal(12, "Paired"))(36)
pal52 <- colorRampPalette(brewer.pal(12, "Paired"))(52)

Import of denoised seurat object:

In [ ]:
seurObj <- LoadH5Seurat('./HTSA_CITE_DSBdenoised.h5seurat')

In [ ]:
seurObj

### Integrated analysis
#### DimRed and batch correction for RNA

Data was previously integrated with scVI, so no PCA for the RNA is available. We first normalise and scale the data, then run a PCA.

In [ ]:
seurObj <- seurObj  %>%
            NormalizeData(assay = 'RNA') %>%
            FindVariableFeatures(assay = 'RNA') %>%
            ScaleData(assay = 'RNA') %>% #regressing out library size might have a negative effect
            RunPCA(assay = 'RNA', npcs = 50, reduction.name = 'pca_rna')

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
ElbowPlot(seurObj, reduction = 'pca_rna', ndims = 50)+labs(title = 'Elbowplot for RNA PCA')

Mathematical PC cutoff calculation based on [this](https://hbctraining.github.io/scRNA-seq_online/lessons/elbow_plot_metric.html).

In [ ]:
#Determine where the % change in variation between 2 subsequent PCs is less than a certain value e.g. 0.1%
PCcutoff <- function(pca, diff){
    var_pc <- pca@stdev/sum(pca@stdev)*100 #calculate % variation associated with each PC
    diffvar_pc <- var_pc[1:length(var_pc)-1] - var_pc[2:length(var_pc)] #calculate difference in variation between subsequent PC
    sort(which(diffvar_pc >diff), decreasing=TRUE)[1]+1 #determine which PC is the last one where the variation is more that x% higher compared to the next PC
}
#pca: the pca slot in the seurat object
#diff: the % difference in variation between two subsequent PCs to be used as cutoff

#Example: determine the last PC where change of % of variation is more than 0.1%:

In [ ]:
dims_rna <- PCcutoff(seurObj@reductions$pca_rna, 0.1)
dims_rna

In [ ]:
seurObj <- RunUMAP(seurObj, reduction = 'pca_rna', assay='RNA', reduction.name = 'umap_rna', dims = 1:dims_rna)

Batch/donor effects are not very strong, but some additional correction may still be beneficial.

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj, reduction = 'umap_rna', group.by = 'sample', cols=pal12)+labs(title='RNA UMAP')

The data set contains data from several donors and experimental batches, so batch correction is required for the RNA. We use [fastMNN](https://rdrr.io/github/LTLA/batchelor/man/reducedMNN.html) from the batchelor package for this purpose, which works on the previously generated PCA. We specify indicidual libraries as batches, but don't specify the merging order.

Of note, several other batch correction tools, e.g. Harmony, have been tested on this data and were found to  result in overcorrection (reduced separation of different mature T lineages etc.). We have made this observation previously on other thymocyte scRNA-seq data sets, suggesting that not all batch correction approaches are equally suitable for this type of developmental data.

In [ ]:
ptm <- proc.time()

MNN_rna <- reducedMNN(seurObj@reductions$pca_rna@cell.embeddings,
                 batch=seurObj$batch, #specify batches
                 #merge.order= unique(seurObj$batch),
                 BPPARAM=MulticoreParam(workers=12), #parallelisation
                 BNPARAM=HnswParam())

proc.time() - ptm

Add batch-corrected PCA as dim reduction.

In [ ]:
seurObj[["mnn_rna"]] <- CreateDimReducObject(embeddings=MNN_rna$corrected,
                                        assay="RNA",
                                        key="mnnrna_")

In [ ]:
seurObj <- RunUMAP(seurObj, reduction = 'mnn_rna', assay='RNA', reduction.name = 'umap_rna_mnn', dims = 1:dims_rna)

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj, reduction = 'umap_rna', group.by = 'sample', cols=pal12)+labs(title='RNA UMAP')
DimPlot(seurObj, reduction = 'umap_rna_mnn', group.by = 'sample', cols = pal12)+labs(title='RNA MNN UMAP')

Donor effects are adjusted in the MNN UMAP but the overall structure is well conserved. Note that even and odd samples correspond to different cell subsets and are thus separate in the UMAP.

#### DimRed and batch correction for ADT

We also carry out scaling and PCA for the ADT data. Note that normalisation was already carried out with dsb and should not be performed again. We use all markers as HVGs (excluding isotype controls).

In [ ]:
VariableFeatures(seurObj, assay = 'ADTdsb') <- rownames(seurObj@assays$ADTdsb@data)[c(1:130,138:150)] #rows 131-137 are isotype controls

In [ ]:
seurObj <- seurObj  %>%
            ScaleData(assay = 'ADTdsb') %>%
            RunPCA(assay = 'ADTdsb', npcs = 50, reduction.name = 'pca_adt')

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)

ElbowPlot(seurObj, reduction = 'pca_adt', ndims = 50)+labs(title = 'Elbowplot for ADT PCA')

In [ ]:
dims_adt <- PCcutoff(seurObj@reductions$pca_adt, 0.1) 
dims_adt

In [ ]:
seurObj <- RunUMAP(seurObj, reduction = 'pca_adt', assay='ADTdsb', reduction.name = 'umap_adt', dims = 1:dims_adt)

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj, reduction = 'umap_adt', group.by = 'sample', cols=pal12)+labs(title='ADT UMAP')

Minor donor effects can be observed for ADT data, so batch correction will be applied for this as well.

In [ ]:
ptm <- proc.time()

MNN_adt <- reducedMNN(seurObj@reductions$pca_adt@cell.embeddings,
                 batch=seurObj$batch, #specify batches
                 #merge.order= unique(seurObj$batch),
                 BPPARAM=MulticoreParam(workers=12), #parallelisation
                 BNPARAM=HnswParam())

proc.time() - ptm

In [ ]:
seurObj[["mnn_adt"]] <- CreateDimReducObject(embeddings=MNN_adt$corrected,
                                        assay="ADTdsb",
                                        key="mnnadt_")

In [ ]:
seurObj <- RunUMAP(seurObj, reduction = 'mnn_adt', assay='ADTdsb', reduction.name = 'umap_adt_mnn', dims = 1:dims_adt)

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj, reduction = 'umap_adt', group.by = 'sample', cols=pal12)+labs(title='ADT UMAP')
DimPlot(seurObj, reduction = 'umap_adt_mnn', group.by = 'sample', cols = pal12)+labs(title='ADT MNN UMAP')

#### DimRed WNN

To get a sense of the UMAP shape and the extend of combined batch effects, we first integrate based on the uncorrected PCAs.

In [ ]:
ptm <- proc.time()

seurObj <- FindMultiModalNeighbors(seurObj, knn.graph.name = 'wknn_pca',
                                   snn.graph.name = 'wsnn_pca',
                                   weighted.nn.name = 'weighted.nn_pca',
                                   reduction.list=list('pca_rna', 'pca_adt'),
                                   dims.list=list(1:dims_rna,1:dims_adt))

proc.time() - ptm

In [ ]:
seurObj <- RunUMAP(seurObj, nn.name = "weighted.nn_pca", reduction.name = "umap_wnn",
                        reduction.key = "wnnUMAP_")

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj, reduction = 'umap_wnn', group.by = 'sample', cols=pal12)+labs(title='WNN UMAP')

In [ ]:
options(repr.plot.width=16, repr.plot.height=6)

FeaturePlot(seurObj, features = c('RNA.weight','ADTdsb.weight'), reduction = 'umap_wnn', order=T)&scale_color_viridis()

Batches are clearly visible, so repeat integration with batch-corrected PCA instead.

In [ ]:
ptm <- proc.time()

seurObj <- FindMultiModalNeighbors(seurObj, knn.graph.name = 'wknn_mnn',
                                   snn.graph.name = 'wsnn_mnn',
                                   weighted.nn.name = 'weighted.nn_mnn',
                                   reduction.list=list('mnn_rna', 'mnn_adt'),
                                   dims.list=list(1:dims_rna,1:dims_adt))

proc.time() - ptm

In [ ]:
seurObj <- RunUMAP(seurObj, nn.name = "weighted.nn_mnn", reduction.name = "umap_wnn_mnn",
                        reduction.key = "wnnmnnUMAP_")

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj, reduction = 'umap_wnn_mnn', group.by = 'sample', cols=pal12)+labs(title='WNN MNN UMAP')

In [ ]:
options(repr.plot.width=16, repr.plot.height=6)

FeaturePlot(seurObj, features = c('RNA.weight','ADTdsb.weight'), reduction = 'umap_wnn_mnn', order=T)&scale_color_viridis()

Reduced batch effects but conserved UMAP structure.

### Additional QC

DN and DP(P) cluster are a little misshapen, potentially some low-quality cells are impacting the structure.

In [ ]:
options(repr.plot.width=20, repr.plot.height=15)

FeaturePlot(seurObj, features = c('doublet_score', "nFeature_RNA", "nFeature_ADT", "pct_counts_ribo", "pct_counts_mt", "cellwise_background_mean", "dsb_technical_component", "dsb_technical_component", "ADTdsb.weight", "RNA.weight"), reduction = 'umap_wnn_mnn', order=T)&scale_color_viridis()


In [ ]:
options(repr.plot.width=20, repr.plot.height=8)

FeaturePlot(seurObj, features = rownames(seurObj@assays$ADT@counts)[131:137], reduction = 'umap_wnn_mnn', order=T, ncol = 4)&scale_color_viridis()

The affected cells have a normal doublet score, mitochondrial percentage etc. explaining why they weren't flagged during the RNA QC. They do however exhibit an exceptionally low number of ADT features but at the same time carry a high weight in the WNN UMAP.

We can try to remove cells with less than 100 detected antibodies (out of 150 total).

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
VlnPlot(seurObj, features = 'nFeature_ADT', group.by = 'sample')&NoLegend()&geom_hline(yintercept = 100)

In [ ]:
seurObj$low_ADT <- (seurObj$nFeature_ADT < 100)

In [ ]:
table(seurObj$low_ADT)

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)

DimPlot(seurObj, group.by = 'low_ADT', reduction = 'umap_wnn_mnn', order=T)

Regressing out the number of detected ABs and similar correction approaches seem to distort the data. Therefore. we remove all cells with ADT < 100 and repeat the integration from scratch.

### Repeat of integration

In [ ]:
seurObj3 <- subset(seurObj, subset=low_ADT==FALSE)

Comparison after/before removal:

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj3, reduction = 'umap_wnn_mnn', group.by = 'sample', cols=pal12)+labs(title='WNN MNN UMAP')
DimPlot(seurObj, reduction = 'umap_wnn_mnn', group.by = 'sample', cols=pal12)+labs(title='WNN MNN UMAP')

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)

FeaturePlot(seurObj3, features = 'nFeature_ADT', reduction = 'umap_wnn_mnn', order=T)+scale_color_viridis()

In [ ]:
seurObj3 <- seurObj3  %>%
            NormalizeData(assay = 'RNA') %>%
            FindVariableFeatures(assay = 'RNA') %>%
            ScaleData(assay = 'RNA') %>%
            RunPCA(assay = 'RNA', npcs = 50, reduction.name = 'pca_rna')

In [ ]:
dims_rna3 <- PCcutoff(seurObj3@reductions$pca_rna, 0.1)
dims_rna3

In [ ]:
ptm <- proc.time()

MNN_rna <- reducedMNN(seurObj3@reductions$pca_rna@cell.embeddings,
                 batch=seurObj3$batch, #specify batches
                 #merge.order= unique(seurObj$batch),
                 BPPARAM=MulticoreParam(workers=12), #parallelisation
                 BNPARAM=HnswParam())

proc.time() - ptm

In [ ]:
seurObj3[["mnn_rna"]] <- CreateDimReducObject(embeddings=MNN_rna$corrected,
                                        assay="RNA",
                                        key="mnnrna_")

In [ ]:
seurObj3 <- seurObj3  %>%
            ScaleData(assay = 'ADTdsb') %>%
            RunPCA(assay = 'ADTdsb', npcs = 50, reduction.name = 'pca_adt')

In [ ]:
dims_adt3 <- PCcutoff(seurObj3@reductions$pca_adt, 0.1) 
dims_adt3

In [ ]:
ptm <- proc.time()

MNN_adt <- reducedMNN(seurObj3@reductions$pca_adt@cell.embeddings,
                 batch=seurObj3$batch, #specify batches
                 #merge.order= unique(seurObj$batch),
                 BPPARAM=MulticoreParam(workers=12), #parallelisation
                 BNPARAM=HnswParam())

proc.time() - ptm

In [ ]:
seurObj3[["mnn_adt"]] <- CreateDimReducObject(embeddings=MNN_adt$corrected,
                                        assay="ADTdsb",
                                        key="mnnadt_")

In [ ]:
ptm <- proc.time()

seurObj3 <- FindMultiModalNeighbors(seurObj3, knn.graph.name = 'wknn_mnn',
                                   snn.graph.name = 'wsnn_mnn',
                                   weighted.nn.name = 'weighted.nn_mnn',
                                   reduction.list=list('mnn_rna', 'mnn_adt'),
                                   dims.list=list(1:dims_rna3,1:dims_adt3))

proc.time() - ptm

In [ ]:
seurObj3 <- RunUMAP(seurObj3, nn.name = "weighted.nn_mnn", reduction.name = "umap_wnn_mnn",n.neighbors = 50,
                        reduction.key = "wnnmnnUMAP_")

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj3, reduction = 'umap_wnn_mnn', group.by = 'sample', cols=pal12)+labs(title='WNN MNN UMAP (post-QC)')
DimPlot(seurObj, reduction = 'umap_wnn_mnn', group.by = 'sample', cols=pal12)+labs(title='WNN MNN UMAP (pre-QC)')

In [ ]:
options(repr.plot.width=7, repr.plot.height=6)

FeaturePlot(seurObj3, features = 'nFeature_ADT', reduction = 'umap_wnn_mnn', order=T)+scale_color_viridis()

In [ ]:
options(repr.plot.width=14, repr.plot.height=6)

FeaturePlot(seurObj3, features = c('ADTdsb.weight', 'RNA.weight'), reduction = 'umap_wnn_mnn', order=T)&scale_color_viridis()

In [ ]:
options(repr.plot.width=16, repr.plot.height=5)

FeaturePlot(seurObj3, features = c('adtdsb_CD4', 'adtdsb_CD8', 'adtdsb_CD34'), ncol=3, reduction = 'umap_wnn_mnn', order=T)&scale_color_viridis()

In [ ]:
SaveH5Seurat(seurObj3, './HTSA_CITE_WNN.h5seurat', overwrite = TRUE)

In [ ]:
seurObj3 <- LoadH5Seurat('./HTSA_CITE_WNN.h5seurat')

## ADT technical artefacts

We had previously noted a subset of cells co-staining for several antibodies, despite not being biologically related or expected to express these markers. This involved only antibodies that had been individually titrated and added to the panel; antibodies in the Human Universal Cocktail were not affected. An in depth investigation revealed this to be most likely caused by antibody aggregation prior to cell staining, resulting in a technical artefact with false-positive cells. To reduce the impact of this artefact on the UMAP structure, it is best to remove the affected cells wherever possible.

Affected markers: CD357, CD370, TCRgd, XCR1, CD199, TCRVa24-Ja18

In [ ]:
options(repr.plot.width=14, repr.plot.height=10)

FeaturePlot(seurObj3, features = c('adtdsb_CD357', 'adtdsb_TCRgd', 'adtdsb_XCR1', 'adtdsb_CD370', 'adtdsb_CD199', "adtdsb_TCRVa24-Ja18"), min.cutoff = 'q05', max.cutoff = 'q95', ncol=3, reduction = 'umap_wnn_mnn', order=T)&scale_color_viridis()

In [ ]:
options(repr.plot.width=12, repr.plot.height=5)

FeatureScatter(seurObj3, feature1 = "adtdsb_CD370", feature2 =  'adtdsb_CD357', group.by = 'anno_CITE_4v5', pt.size = 2)
FeatureScatter(seurObj3, feature1 = "adtdsb_TCRgd", feature2 =  'adtdsb_CD357', group.by = 'anno_CITE_4v5', pt.size = 2)
FeatureScatter(seurObj3, feature1 = "adtdsb_TCRgd", feature2 =  'adtdsb_XCR1', group.by = 'anno_CITE_4v5', pt.size = 2)
FeatureScatter(seurObj3, feature1 = "adtdsb_TCRVa24-Ja18", feature2 = 'adtdsb_TCRgd', group.by = 'anno_CITE_4v5', pt.size = 2)

We can set specific thresholds to tag cells affected by the artefact.

In [ ]:
cells_artefact <- WhichCells(seurObj3, expression = (adtdsb_TCRgd > 30 & (adtdsb_CD370 > 30 | adtdsb_CD357 > 30 | adtdsb_XCR1 >50 | `adtdsb_TCRVa24-Ja18` >50)))

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj3, reduction = 'umap_wnn_mnn', cells.highlight = cells_artefact, order=T,sizes.highlight = 1.5, raster = FALSE)

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj3, reduction = 'umap_adt_mnn', cells.highlight = cells_artefact, order=T,sizes.highlight = 1.5, raster = FALSE)

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj3, reduction = 'umap_rna_mnn', cells.highlight = cells_artefact, order=T, sizes.highlight = 1.5, raster = FALSE)

Note that the cells cluster together in the ADT and partially in the WNN UMAP but not in the RNA UMAP, confirming that these are 'similar' in their surface profile but completley distinct in their gene expression/identity.

We therefore remove these cells from the data set. This affects 466 cells in total.

In [ ]:
seurObj4 <- subset(seurObj3, cells = WhichCells(seurObj3, expression = (adtdsb_TCRgd > 30 & (adtdsb_CD370 > 30 | adtdsb_CD357 > 30 | adtdsb_CD199 > 25)) | adtdsb_XCR1 >50 | adtdsb_CD370 >50 | adtdsb_CD357 > 100), invert=TRUE)

In [ ]:
length(colnames(seurObj3))-length(colnames(seurObj4))

We then repeat the full batch-correction/integration procedure once more.

In [ ]:
seurObj4 <- seurObj4  %>%
            NormalizeData(assay = 'RNA') %>%
            FindVariableFeatures(assay = 'RNA') %>%
            ScaleData(assay = 'RNA') %>%
            RunPCA(assay = 'RNA', npcs = 50, reduction.name = 'pca_rna')

In [ ]:
dims_rna4 <- PCcutoff(seurObj4@reductions$pca_rna, 0.1)
dims_rna4

In [ ]:
ptm <- proc.time()

MNN_rna <- reducedMNN(seurObj4@reductions$pca_rna@cell.embeddings,
                 batch=seurObj4$batch, #specify batches
                 #merge.order= unique(seurObj$batch),
                 BPPARAM=MulticoreParam(workers=12), #parallelisation
                 BNPARAM=HnswParam())

proc.time() - ptm

In [ ]:
seurObj4[["mnn_rna"]] <- CreateDimReducObject(embeddings=MNN_rna$corrected,
                                        assay="RNA",
                                        key="mnnrna_")

In [ ]:
seurObj4 <- seurObj4  %>%
            ScaleData(assay = 'ADTdsb') %>%
            RunPCA(assay = 'ADTdsb', npcs = 50, reduction.name = 'pca_adt')

In [ ]:
dims_adt4 <- PCcutoff(seurObj4@reductions$pca_adt, 0.1) 
dims_adt4

In [ ]:
ptm <- proc.time()

MNN_adt <- reducedMNN(seurObj4@reductions$pca_adt@cell.embeddings,
                 batch=seurObj4$batch, #specify batches
                 #merge.order= unique(seurObj$batch),
                 BPPARAM=MulticoreParam(workers=12), #parallelisation
                 BNPARAM=HnswParam())

proc.time() - ptm

In [ ]:
seurObj4[["mnn_adt"]] <- CreateDimReducObject(embeddings=MNN_adt$corrected,
                                        assay="ADTdsb",
                                        key="mnnadt_")

In [ ]:
ptm <- proc.time()

seurObj4 <- FindMultiModalNeighbors(seurObj4, knn.graph.name = 'wknn_mnn',
                                   snn.graph.name = 'wsnn_mnn',
                                   weighted.nn.name = 'weighted.nn_mnn',
                                   reduction.list=list('mnn_rna', 'mnn_adt'),
                                   dims.list=list(1:dims_rna4,1:dims_adt4))

proc.time() - ptm

In [ ]:
seurObj4 <- RunUMAP(seurObj4, nn.name = "weighted.nn_mnn", reduction.name = "umap_wnn_mnn",n.neighbors = 50,
                        reduction.key = "wnnmnnUMAP_")

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
DimPlot(seurObj3, reduction = 'umap_wnn_mnn', group.by = 'sample', cols=pal12)+labs(title='WNN MNN UMAP (post-QC1)')
DimPlot(seurObj4, reduction = 'umap_wnn_mnn', group.by = 'sample', cols=pal12)+labs(title='WNN MNN UMAP (post-QC2)')

In [ ]:
options(repr.plot.width=14, repr.plot.height=6)

FeaturePlot(seurObj4, features = c('ADTdsb.weight', 'RNA.weight'), reduction = 'umap_wnn_mnn', order=T)&scale_color_viridis()

In [ ]:
options(repr.plot.width=14, repr.plot.height=10)

FeaturePlot(seurObj4, features = c('adtdsb_CD357', 'adtdsb_TCRgd', 'adtdsb_XCR1', 'adtdsb_CD370', 'adtdsb_CD199', "adtdsb_TCRVa24-Ja18"), min.cutoff = 'q05', max.cutoff = 'q95', ncol=3, reduction = 'umap_wnn_mnn', order=T)&scale_color_viridis()

Note that cells that are true-positive for the affected markers are still retained in the data set, e.g. CD357+ Tregs and gd T cells. This is because false-positive thresholding was done on a combination of markers, which are not normally co-expressed. 

In [ ]:
options(repr.plot.width=16, repr.plot.height=5)

FeaturePlot(seurObj4, features = c('adtdsb_CD34', 'adtdsb_CD4', 'adtdsb_CD31'), ncol=3, reduction = 'umap_wnn_mnn', order=T)&scale_color_viridis()

In [ ]:
seurObj4 <- RunSPCA(seurObj4, assay='RNA', graph='wsnn_mnn')

In [ ]:
SaveH5Seurat(seurObj4, './HTSA_CITE_WNN_corr.h5seurat', overwrite = TRUE)